In [1]:
from bokeh.plotting import figure, show
from bokeh.embed import components
from bokeh.models import ColumnDataSource, FactorRange, HoverTool
from bokeh.transform import  factor_cmap



import pandas as pd
from pathlib import Path

In [2]:
data_path = Path("../../../Resources/RileyAnalysis/emdat_population.json")

disaster_population_df = pd.read_json(data_path)

In [3]:
disaster_population_df.head()

,Disaster #,Subgroup,Type,Subtype,Country,ISO,Region,Geolocation,Magnitude,Lat,...,Year,Month,Deaths,Injuries,Total Affected,Insured Damage (Adjusted),Total Damage (Adjusted),population,density,area
0,1999-9388,Climatological,Drought,Drought,Djibouti,DJI,Africa,Kibungo,NaN,-2.135743,...,2001,6.0,NaN,NaN,100000.0,NaN,NaN,958920,41.368421,23200
1,1999-9388,Climatological,Drought,Drought,Djibouti,DJI,Africa,Rugombo,NaN,-2.821939,...,2001,6.0,NaN,NaN,100000.0,NaN,NaN,958920,41.368421,23200
2,1999-9388,Climatological,Drought,Drought,Djibouti,DJI,Africa,Rumonge,NaN,-3.962206,...,2001,6.0,NaN,NaN,100000.0,NaN,NaN,958920,41.368421,23200
3,1999-9388,Climatological,Drought,Drought,Djibouti,DJI,Africa,Buganda,NaN,-2.990570,...,2001,6.0,NaN,NaN,100000.0,NaN,NaN,958920,41.368421,23200
4,1999-9388,Climatological,Drought,Drought,Djibouti,DJI,Africa,Gihanga,NaN,-3.197334,...,2001,6.0,NaN,NaN,100000.0,NaN,NaN,958920,41.368421,23200


In [4]:
#number of unique disaster id's in the dataset
disaster_population_df['Disaster #'].nunique()

5087

In [5]:
country_df = disaster_population_df[['Country', 'Type', 'Disaster #']].groupby(['Country', 'Type']).nunique()

print(country_df.shape)
country_df.head(10)

(648, 1)


Disaster #
Country     Type                           
Afghanistan Drought                       4
            Earthquake                   16
            Extreme temperature           5
            Flood                        62
            Mass movement (wet)          17
            Storm                         6
Albania     Earthquake                    1
            Extreme temperature           4
            Flood                        11
            Storm                         2

In [6]:
#for each (disaster type for each?) country in my countries of interest (can choose to limit) we want to see how the following metrics compare:

#number of disaster occurneces per population

#number of disaster occurences per population density

#number of disaster occurneces per surface area

In [7]:
alt_country_df = disaster_population_df[['Country', 'Disaster #']].groupby(['Country'], as_index=False).nunique()

alt_country_df.columns = ['Country', 'disaster_count']

print(alt_country_df.shape)
alt_country_df.head(10)

(192, 2)


,Country,disaster_count
0,Afghanistan,110
1,Albania,18
2,Algeria,41
3,American Samoa,4
4,Angola,42
5,Anguilla,1
6,Antigua and Barbuda,2
7,Argentina,61
8,Armenia,6
9,Australia,74


In [8]:
temp_df = disaster_population_df[['Country', 'population', 'density', 'area']].groupby('Country', as_index=False).first()
temp_df.head()

,Country,population,density,area
0,Afghanistan,37172386,56.937760,652090
1,Albania,2866376,104.612263,28748
2,Algeria,42228429,17.730075,2381741
3,American Samoa,55465,277.325000,199
4,Angola,30809762,24.713052,1246700


In [9]:
alt_country_df = alt_country_df.merge(temp_df, on='Country')
print(alt_country_df.shape)
alt_country_df.head()

(192, 5)


,Country,disaster_count,population,density,area
0,Afghanistan,110,37172386,56.937760,652090
1,Albania,18,2866376,104.612263,28748
2,Algeria,41,42228429,17.730075,2381741
3,American Samoa,4,55465,277.325000,199
4,Angola,42,30809762,24.713052,1246700


In [10]:
alt_country_df['disaster_per_capita'] = alt_country_df['disaster_count']/alt_country_df['population']

alt_country_df['disaster_per_square_km'] = alt_country_df['disaster_count']/alt_country_df['area']

alt_country_df.head()

,Country,disaster_count,population,density,area,disaster_per_capita,disaster_per_square_km
0,Afghanistan,110,37172386,56.937760,652090,2.959186e-06,0.000169
1,Albania,18,2866376,104.612263,28748,6.279707e-06,0.000626
2,Algeria,41,42228429,17.730075,2381741,9.709099e-07,0.000017
3,American Samoa,4,55465,277.325000,199,7.211755e-05,0.020101
4,Angola,42,30809762,24.713052,1246700,1.363204e-06,0.000034


In [11]:
final_country_pop_disaster_df = alt_country_df.copy()

final_country_pop_disaster_df = final_country_pop_disaster_df.sort_values('disaster_per_capita', ascending=False)

final_country_pop_disaster_df.head()

,Country,disaster_count,population,density,area,disaster_per_capita,disaster_per_square_km
170,Tokelau,1,1411,115.00,12,0.000709,0.083333
122,Niue,1,1624,5.35,260,0.000616,0.003846
176,Tuvalu,2,11508,383.60,26,0.000174,0.076923
111,Montserrat,1,5900,50.00,102,0.000169,0.009804
143,Saint Helena,1,6600,37.30,314,0.000152,0.003185


In [12]:
country_visual_one_df = final_country_pop_disaster_df.head(25)

country_visual_one_df

,Country,disaster_count,population,density,area,disaster_per_capita,disaster_per_square_km
170,Tokelau,1,1411,115.000000,12,0.000709,0.083333
122,Niue,1,1624,5.350000,260,0.000616,0.003846
176,Tuvalu,2,11508,383.600000,26,0.000174,0.076923
111,Montserrat,1,5900,50.000000,102,0.000169,0.009804
143,Saint Helena,1,6600,37.300000,314,0.000152,0.003185
175,Turks and Caicos Islands,5,37665,39.647368,430,0.000133,0.011628
128,Palau,2,17907,38.928261,459,0.000112,0.004357
31,Cayman Islands,7,64174,267.391667,264,0.000109,0.026515
171,Tonga,10,103197,143.329167,650,0.000097,0.015385
124,Northern Mariana Islands,5,56882,123.656522,464,0.000088,0.010776


In [13]:
#produce the countries by disasters per capita

# colors = ['#ffee65', '#fdcce5', '#fd7f6f', '#7eb0d5', '#bd7ebe', '#8bd3c7', '#b2e061', '#ffb55a']

alt_colors = ["#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95", "#17A398", "#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95"]


x_values = country_visual_one_df['Country']
y_values = country_visual_one_df['disaster_per_capita']*100000


p = figure(x_range=x_values, width= 1200, height=500, title="Top 25 Countries by Disasters Per Capita", x_axis_label='Country', y_axis_label='Disasters Per Million People', toolbar_location=None, tools="")

p.vbar(x=x_values, top=y_values, width=0.9, color = alt_colors)
p.xgrid.grid_line_color = None
p.y_range.start = 0

#Grabbed the following chunk from Connor:
#Adjust x axis lables for readability
p.xaxis.major_label_orientation = 1.5

#save Javascript code and html code and show plot in browser
js_plot_one, html_plot_one = components(p)
show(p)

In [14]:
print(js_plot_one)

    <script type="text/javascript">
        (function() {
  const fn = function() {
    Bokeh.safely(function() {
      (function(root) {
        function embed_document(root) {
        const docs_json = '{"6baf77f0-c256-4e08-993d-c64d5040f0e0":{"version":"3.3.0","title":"Bokeh Application","roots":[{"type":"object","name":"Figure","id":"p1001","attributes":{"width":1200,"height":500,"x_range":{"type":"object","name":"FactorRange","id":"p1011","attributes":{"factors":["Tokelau","Niue","Tuvalu","Montserrat","Saint Helena","Turks and Caicos Islands","Palau","Cayman Islands","Tonga","Northern Mariana Islands","Saint Vincent and the Grenadines","American Samoa","Dominica","Anguilla","Vanuatu","Cook Islands","Saint Lucia","Seychelles","Bahamas","Marshall Islands","Samoa","Belize","Grenada","Solomon Islands","Antigua and Barbuda"]}},"y_range":{"type":"object","name":"DataRange1d","id":"p1003","attributes":{"start":0}},"x_scale":{"type":"object","name":"CategoricalScale","id":"p1012"},"y_scal

In [15]:
#now lets take out entries that have only 1 disaster 

cleaned_country_visual_one_df = final_country_pop_disaster_df.copy()
cleaned_country_visual_one_df = cleaned_country_visual_one_df.drop(cleaned_country_visual_one_df[cleaned_country_visual_one_df.disaster_count == 1].index).head(25)

cleaned_country_visual_one_df

,Country,disaster_count,population,density,area,disaster_per_capita,disaster_per_square_km
176,Tuvalu,2,11508,383.600000,26,0.000174,0.076923
175,Turks and Caicos Islands,5,37665,39.647368,430,0.000133,0.011628
128,Palau,2,17907,38.928261,459,0.000112,0.004357
31,Cayman Islands,7,64174,267.391667,264,0.000109,0.026515
171,Tonga,10,103197,143.329167,650,0.000097,0.015385
124,Northern Mariana Islands,5,56882,123.656522,464,0.000088,0.010776
146,Saint Vincent and the Grenadines,8,110210,282.589744,388,0.000073,0.020619
3,American Samoa,4,55465,277.325000,199,0.000072,0.020101
50,Dominica,5,71625,95.500000,751,0.000070,0.006658
186,Vanuatu,19,292680,24.009844,12189,0.000065,0.001559


In [16]:
#same visual but with the cleaned data frame

alt_colors = ["#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95", "#17A398", "#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95"]


x_values = cleaned_country_visual_one_df['Country']
y_values = cleaned_country_visual_one_df['disaster_per_capita']*100000

p = figure(x_range=x_values, width= 1200, height=500, title="Top 25 Countries by Disasters Per Capita (minimum 2 Disasters)", x_axis_label='Country', y_axis_label='Disasters Per Million People', toolbar_location=None, tools="")

p.vbar(x=x_values, top=y_values, width=0.9, color = alt_colors)
p.xgrid.grid_line_color = None
p.y_range.start = 0

#Grabbed the following chunk from Connor:
#Adjust x axis lables for readability
p.xaxis.major_label_orientation = 1.5

#save Javascript code and html code and show plot in browser
js_plot_two, html_plot_two = components(p)
show(p)

In [17]:
print(js_plot_two)

    <script type="text/javascript">
        (function() {
  const fn = function() {
    Bokeh.safely(function() {
      (function(root) {
        function embed_document(root) {
        const docs_json = '{"256e81b3-95a9-4c2c-bfdb-03211cbcebcd":{"version":"3.3.0","title":"Bokeh Application","roots":[{"type":"object","name":"Figure","id":"p1036","attributes":{"width":1200,"height":500,"x_range":{"type":"object","name":"FactorRange","id":"p1046","attributes":{"factors":["Tuvalu","Turks and Caicos Islands","Palau","Cayman Islands","Tonga","Northern Mariana Islands","Saint Vincent and the Grenadines","American Samoa","Dominica","Vanuatu","Saint Lucia","Seychelles","Bahamas","Marshall Islands","Samoa","Belize","Grenada","Solomon Islands","Antigua and Barbuda","Barbados","Kiribati","Guam","Martinique","Comoros","North Macedonia"]}},"y_range":{"type":"object","name":"DataRange1d","id":"p1038","attributes":{"start":0}},"x_scale":{"type":"object","name":"CategoricalScale","id":"p1047"},"y_scale

In [18]:
#we want the bottom 25

final_country_pop_disaster_df = final_country_pop_disaster_df.sort_values('disaster_per_capita', ascending=True)
country_visual_bottom_df = final_country_pop_disaster_df.head(25)

country_visual_bottom_df

,Country,disaster_count,population,density,area,disaster_per_capita,disaster_per_square_km
185,Uzbekistan,3,32955400,77.469205,447400,9.103212e-08,0.000007
180,United Arab Emirates,1,9630959,135.609110,83600,1.038318e-07,0.000012
53,Egypt,13,98423595,98.873469,1001449,1.320821e-07,0.000013
174,Turkmenistan,1,5850908,12.450595,488100,1.709136e-07,0.000002
59,Finland,1,5515525,18.156856,338145,1.813064e-07,0.000003
89,Jordan,2,9956011,112.142498,88946,2.008837e-07,0.000022
121,Nigeria,43,195874740,215.064989,923768,2.195281e-07,0.000047
80,India,301,1352617328,454.938073,3287263,2.225315e-07,0.000092
83,Iraq,11,38433600,88.530570,438317,2.862079e-07,0.000025
163,Sweden,3,10175214,25.001043,449964,2.948341e-07,0.000007


In [19]:
alt_colors = ["#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95", "#17A398", "#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95"]


x_values = country_visual_bottom_df['Country']
y_values = country_visual_bottom_df['disaster_per_capita']*100000

p = figure(x_range=x_values, width= 1200, height=500, title="Bottom 25 Countries by Disasters Per Capita", x_axis_label='Country', y_axis_label='Disasters Per Million People', toolbar_location=None, tools="")

p.vbar(x=x_values, top=y_values, width=0.9, color = alt_colors)
p.xgrid.grid_line_color = None
p.y_range.start = 0

#Grabbed the following chunk from Connor:
#Adjust x axis lables for readability
p.xaxis.major_label_orientation = 1.5

#save Javascript code and html code and show plot in browser
js_plot_three, html_plot_three = components(p)
show(p)

In [20]:
print(js_plot_three)

    <script type="text/javascript">
        (function() {
  const fn = function() {
    Bokeh.safely(function() {
      (function(root) {
        function embed_document(root) {
        const docs_json = '{"02a2a2bf-9f43-4a3c-89f9-32d044275c99":{"version":"3.3.0","title":"Bokeh Application","roots":[{"type":"object","name":"Figure","id":"p1071","attributes":{"width":1200,"height":500,"x_range":{"type":"object","name":"FactorRange","id":"p1081","attributes":{"factors":["Uzbekistan","United Arab Emirates","Egypt","Turkmenistan","Finland","Jordan","Nigeria","India","Iraq","Sweden","Qatar","China","Ethiopia","C\\u00f4te d\\u2019Ivoire","Syrian Arab Republic","Brazil","Eritrea","Pakistan","Ukraine","Germany","Ghana","Cameroon","Saudi Arabia","Spain","Tunisia"]}},"y_range":{"type":"object","name":"DataRange1d","id":"p1073","attributes":{"start":0}},"x_scale":{"type":"object","name":"CategoricalScale","id":"p1082"},"y_scale":{"type":"object","name":"LinearScale","id":"p1083"},"title":{"type":

In [21]:
#minimum two disasters

country_visual_bottom_df = final_country_pop_disaster_df.copy()
country_visual_bottom_df = country_visual_bottom_df.drop(country_visual_bottom_df[country_visual_bottom_df.disaster_count == 1].index).head(25)

country_visual_bottom_df

,Country,disaster_count,population,density,area,disaster_per_capita,disaster_per_square_km
185,Uzbekistan,3,32955400,77.469205,447400,9.103212e-08,0.000007
53,Egypt,13,98423595,98.873469,1001449,1.320821e-07,0.000013
89,Jordan,2,9956011,112.142498,88946,2.008837e-07,0.000022
121,Nigeria,43,195874740,215.064989,923768,2.195281e-07,0.000047
80,India,301,1352617328,454.938073,3287263,2.225315e-07,0.000092
83,Iraq,11,38433600,88.530570,438317,2.862079e-07,0.000025
163,Sweden,3,10175214,25.001043,449964,2.948341e-07,0.000007
35,China,532,1392730000,148.348833,9572900,3.819836e-07,0.000056
58,Ethiopia,42,109224559,109.224559,1104300,3.845289e-07,0.000038
46,Côte d’Ivoire,10,25069229,78.834053,322463,3.988954e-07,0.000031


In [22]:
alt_colors = ["#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95", "#17A398", "#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95"]


x_values = country_visual_bottom_df['Country']
y_values = country_visual_bottom_df['disaster_per_capita']*100000

p = figure(x_range=x_values, width= 1200, height=500, title="Bottom 25 Countries by Disasters Per Capita (minimum 2 Disasters)", x_axis_label='Country', y_axis_label='Disasters Per Million People', toolbar_location=None, tools="")

p.vbar(x=x_values, top=y_values, width=0.9, color = alt_colors)
p.xgrid.grid_line_color = None
p.y_range.start = 0

#Grabbed the following chunk from Connor:
#Adjust x axis lables for readability
p.xaxis.major_label_orientation = 1.5

#save Javascript code and html code and show plot in browser
js_plot_four, html_plot_four = components(p)
show(p)

In [23]:
print(js_plot_four)

    <script type="text/javascript">
        (function() {
  const fn = function() {
    Bokeh.safely(function() {
      (function(root) {
        function embed_document(root) {
        const docs_json = '{"d75f6ce0-b3c5-42ef-9c3f-0f2241d114e2":{"version":"3.3.0","title":"Bokeh Application","roots":[{"type":"object","name":"Figure","id":"p1106","attributes":{"width":1200,"height":500,"x_range":{"type":"object","name":"FactorRange","id":"p1116","attributes":{"factors":["Uzbekistan","Egypt","Jordan","Nigeria","India","Iraq","Sweden","China","Ethiopia","C\\u00f4te d\\u2019Ivoire","Syrian Arab Republic","Brazil","Eritrea","Pakistan","Ukraine","Germany","Ghana","Cameroon","Saudi Arabia","Spain","Tunisia","United Kingdom of Great Britain and Northern Ireland","Sudan","Morocco","United Republic of Tanzania"]}},"y_range":{"type":"object","name":"DataRange1d","id":"p1108","attributes":{"start":0}},"x_scale":{"type":"object","name":"CategoricalScale","id":"p1117"},"y_scale":{"type":"object","nam

Now working on visuals for disasters per square kilometer

In [24]:
final_country_pop_disaster_df = final_country_pop_disaster_df.sort_values('disaster_per_square_km', ascending=False)

final_country_pop_disaster_df.head()

,Country,disaster_count,population,density,area,disaster_per_capita,disaster_per_square_km
37,"China, Macao Special Administrative Region",2,631636,20777.500260,18,0.000003,0.111111
170,Tokelau,1,1411,115.000000,12,0.000709,0.083333
176,Tuvalu,2,11508,383.600000,26,0.000174,0.076923
31,Cayman Islands,7,64174,267.391667,264,0.000109,0.026515
146,Saint Vincent and the Grenadines,8,110210,282.589744,388,0.000073,0.020619


In [25]:
plot_five_df = final_country_pop_disaster_df.head(25)

In [26]:
#now we want visuals for disasters per square kilometer


alt_colors = ["#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95", "#17A398", "#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95"]


x_values = plot_five_df['Country']
y_values = plot_five_df['disaster_per_square_km']


p = figure(x_range=x_values, width= 1200, height=500, title="Top 25 Countries by Disasters Per Size of Country (km^2)", x_axis_label='Country', y_axis_label='Disasters Per km^2', toolbar_location=None, tools="")

p.vbar(x=x_values, top=y_values, width=0.9, color = alt_colors)
p.xgrid.grid_line_color = None
p.y_range.start = 0

#Grabbed the following chunk from Connor:
#Adjust x axis lables for readability
p.xaxis.major_label_orientation = 1.5

#save Javascript code and html code and show plot in browser
js_plot_five, html_plot_five = components(p)
show(p)

In [27]:
print(js_plot_five)

    <script type="text/javascript">
        (function() {
  const fn = function() {
    Bokeh.safely(function() {
      (function(root) {
        function embed_document(root) {
        const docs_json = '{"e864a7c3-064b-490e-916a-dc2787f8f84f":{"version":"3.3.0","title":"Bokeh Application","roots":[{"type":"object","name":"Figure","id":"p1141","attributes":{"width":1200,"height":500,"x_range":{"type":"object","name":"FactorRange","id":"p1151","attributes":{"factors":["China, Macao Special Administrative Region","Tokelau","Tuvalu","Cayman Islands","Saint Vincent and the Grenadines","American Samoa","Tonga","Saint Lucia","Barbados","Turks and Caicos Islands","Marshall Islands","Northern Mariana Islands","Anguilla","Montserrat","Seychelles","Grenada","China, Hong Kong Special Administrative Region","Dominica","Antigua and Barbuda","Palau","Comoros","Cook Islands","Niue","Saint Kitts and Nevis","Guam"]}},"y_range":{"type":"object","name":"DataRange1d","id":"p1143","attributes":{"start":0}

In [28]:
plot_six_df = final_country_pop_disaster_df.copy()
plot_six_df = plot_six_df.drop(plot_six_df[plot_six_df.disaster_count == 1].index).head(25)

plot_six_df

,Country,disaster_count,population,density,area,disaster_per_capita,disaster_per_square_km
37,"China, Macao Special Administrative Region",2,631636,20777.500260,18,0.000003,0.111111
176,Tuvalu,2,11508,383.600000,26,0.000174,0.076923
31,Cayman Islands,7,64174,267.391667,264,0.000109,0.026515
146,Saint Vincent and the Grenadines,8,110210,282.589744,388,0.000073,0.020619
3,American Samoa,4,55465,277.325000,199,0.000072,0.020101
171,Tonga,10,103197,143.329167,650,0.000097,0.015385
145,Saint Lucia,8,181889,298.178689,622,0.000044,0.012862
14,Barbados,5,286641,666.606977,430,0.000017,0.011628
175,Turks and Caicos Islands,5,37665,39.647368,430,0.000133,0.011628
105,Marshall Islands,2,58413,324.516667,181,0.000034,0.011050


In [29]:
#now we want visuals for disasters per square kilometer


alt_colors = ["#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95", "#17A398", "#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95"]


x_values = plot_six_df['Country']
y_values = plot_six_df['disaster_per_square_km']


p = figure(x_range=x_values, width= 1200, height=500, title="Top 25 Countries by Disasters Per Size of Country (km^2) (minimum two disasters)", x_axis_label='Country', y_axis_label='Disasters Per km^2', toolbar_location=None, tools="")

p.vbar(x=x_values, top=y_values, width=0.9, color = alt_colors)
p.xgrid.grid_line_color = None
p.y_range.start = 0

#Grabbed the following chunk from Connor:
#Adjust x axis lables for readability
p.xaxis.major_label_orientation = 1.5

#save Javascript code and html code and show plot in browser
js_plot_six, html_plot_six = components(p)
show(p)

In [30]:
print(js_plot_six)

    <script type="text/javascript">
        (function() {
  const fn = function() {
    Bokeh.safely(function() {
      (function(root) {
        function embed_document(root) {
        const docs_json = '{"4c3f3a08-9ffe-4718-95e4-1bcc224777ca":{"version":"3.3.0","title":"Bokeh Application","roots":[{"type":"object","name":"Figure","id":"p1176","attributes":{"width":1200,"height":500,"x_range":{"type":"object","name":"FactorRange","id":"p1186","attributes":{"factors":["China, Macao Special Administrative Region","Tuvalu","Cayman Islands","Saint Vincent and the Grenadines","American Samoa","Tonga","Saint Lucia","Barbados","Turks and Caicos Islands","Marshall Islands","Northern Mariana Islands","Seychelles","Grenada","China, Hong Kong Special Administrative Region","Dominica","Antigua and Barbuda","Palau","Comoros","Guam","Martinique","Kiribati","Haiti","Samoa","Guadeloupe","Jamaica"]}},"y_range":{"type":"object","name":"DataRange1d","id":"p1178","attributes":{"start":0}},"x_scale":{"typ

In [31]:
#now we want to do bottom 25 countries by disasters per size of country

final_country_pop_disaster_df = final_country_pop_disaster_df.sort_values('disaster_per_square_km', ascending=True)

final_country_pop_disaster_df.head()

,Country,disaster_count,population,density,area,disaster_per_capita,disaster_per_square_km
174,Turkmenistan,1,5850908,12.450595,488100,1.709136e-07,0.000002
59,Finland,1,5515525,18.156856,338145,1.813064e-07,0.000003
30,Canada,45,37057765,4.075308,9970610,1.214320e-06,0.000005
163,Sweden,3,10175214,25.001043,449964,2.948341e-07,0.000007
185,Uzbekistan,3,32955400,77.469205,447400,9.103212e-08,0.000007


In [32]:
plot_seven_df = final_country_pop_disaster_df.head(25)

In [33]:

alt_colors = ["#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95", "#17A398", "#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95"]


x_values = plot_seven_df['Country']
y_values = plot_seven_df['disaster_per_square_km']


p = figure(x_range=x_values, width= 1200, height=500, title="Bottom 25 Countries by Disasters Per Size of Country (km^2)", x_axis_label='Country', y_axis_label='Disasters Per km^2', toolbar_location=None, tools="")

p.vbar(x=x_values, top=y_values, width=0.9, color = alt_colors)
p.xgrid.grid_line_color = None
p.y_range.start = 0

#Grabbed the following chunk from Connor:
#Adjust x axis lables for readability
p.xaxis.major_label_orientation = 1.5

#save Javascript code and html code and show plot in browser
js_plot_seven, html_plot_seven = components(p)
show(p)

In [34]:
print(js_plot_seven)

    <script type="text/javascript">
        (function() {
  const fn = function() {
    Bokeh.safely(function() {
      (function(root) {
        function embed_document(root) {
        const docs_json = '{"0e0f03c8-11cd-4ad5-8ef8-c38fd836e081":{"version":"3.3.0","title":"Bokeh Application","roots":[{"type":"object","name":"Figure","id":"p1211","attributes":{"width":1200,"height":500,"x_range":{"type":"object","name":"FactorRange","id":"p1221","attributes":{"factors":["Turkmenistan","Finland","Canada","Sweden","Uzbekistan","Saudi Arabia","Mongolia","Australia","Brazil","French Guiana","Chad","United Arab Emirates","Suriname","Norway","Egypt","Sudan","South Sudan","Gabon","Botswana","Mauritania","Algeria","Mali","Namibia","Zambia","Argentina"]}},"y_range":{"type":"object","name":"DataRange1d","id":"p1213","attributes":{"start":0}},"x_scale":{"type":"object","name":"CategoricalScale","id":"p1222"},"y_scale":{"type":"object","name":"LinearScale","id":"p1223"},"title":{"type":"object","nam

In [35]:
#now the same but we remove only 1 disaster:

plot_eight_df = final_country_pop_disaster_df.copy()
plot_eight_df = plot_eight_df.drop(plot_eight_df[plot_eight_df.disaster_count == 1].index).head(25)

plot_eight_df

,Country,disaster_count,population,density,area,disaster_per_capita,disaster_per_square_km
30,Canada,45,37057765,4.075308,9970610,1.214320e-06,0.000005
163,Sweden,3,10175214,25.001043,449964,2.948341e-07,0.000007
185,Uzbekistan,3,32955400,77.469205,447400,9.103212e-08,0.000007
148,Saudi Arabia,19,33699947,15.676654,2149690,5.637991e-07,0.000009
110,Mongolia,14,3170208,2.040609,1566500,4.416114e-06,0.000009
9,Australia,74,24982688,3.249129,7741220,2.962051e-06,0.000010
23,Brazil,91,209469333,25.061716,8547403,4.344311e-07,0.000011
33,Chad,15,15477751,12.291734,1284000,9.691330e-07,0.000012
162,Suriname,2,575991,3.692250,163265,3.472276e-06,0.000012
125,Norway,4,5311916,14.554920,323877,7.530240e-07,0.000012


In [36]:

alt_colors = ["#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95", "#17A398", "#2679ad","#fd7f6f","#ffee65","#63bb3b","#ffb55a","#7eb0d5","#bd7ebe","#b2e061","#fdcce5","#8bd3c7", "#F78F2D", "#CD9C95"]


x_values = plot_eight_df['Country']
y_values = plot_eight_df['disaster_per_square_km']


p = figure(x_range=x_values, width= 1200, height=500, title="Bottom 25 Countries by Disasters Per Size of Country (km^2) (minimum 2 disasters)", x_axis_label='Country', y_axis_label='Disasters Per km^2', toolbar_location=None, tools="")

p.vbar(x=x_values, top=y_values, width=0.9, color = alt_colors)
p.xgrid.grid_line_color = None
p.y_range.start = 0

#Grabbed the following chunk from Connor:
#Adjust x axis lables for readability
p.xaxis.major_label_orientation = 1.5

#save Javascript code and html code and show plot in browser
js_plot_eight, html_plot_eight = components(p)
show(p)

In [37]:
print(js_plot_eight)

    <script type="text/javascript">
        (function() {
  const fn = function() {
    Bokeh.safely(function() {
      (function(root) {
        function embed_document(root) {
        const docs_json = '{"ed57cf0f-8994-428d-816e-566a43ad27e1":{"version":"3.3.0","title":"Bokeh Application","roots":[{"type":"object","name":"Figure","id":"p1246","attributes":{"width":1200,"height":500,"x_range":{"type":"object","name":"FactorRange","id":"p1256","attributes":{"factors":["Canada","Sweden","Uzbekistan","Saudi Arabia","Mongolia","Australia","Brazil","Chad","Suriname","Norway","Egypt","Sudan","South Sudan","Gabon","Botswana","Mauritania","Algeria","Mali","Namibia","Zambia","Argentina","Niger","Jordan","Iraq","Venezuela (Bolivarian Republic of)"]}},"y_range":{"type":"object","name":"DataRange1d","id":"p1248","attributes":{"start":0}},"x_scale":{"type":"object","name":"CategoricalScale","id":"p1257"},"y_scale":{"type":"object","name":"LinearScale","id":"p1258"},"title":{"type":"object","name":

Now scatter plots that look at disaster count vs population and disaster count vs area

In [38]:
from scipy.stats import linregress

In [39]:
#using the final_country_pop_disaster_df

# Calculate regression line
slope, intercept, r_value, p_value, std_err = linregress(final_country_pop_disaster_df['population'], final_country_pop_disaster_df['disaster_count'])
regression_line_x = final_country_pop_disaster_df['population']
regression_line_y = slope * regression_line_x + intercept



x_values = final_country_pop_disaster_df['population']
y_values = final_country_pop_disaster_df['disaster_count']
country_names = final_country_pop_disaster_df['Country']

# source = ColumnDataSource(data=dict(x=x_values, y=y_values, desc=country_names))
# hover = HoverTool(tooltops=[("index","$index"),(,),(,)])

p = figure( width= 1200, height=500, title="Disasters in Countries by Population", x_axis_label='Population', y_axis_label='# Disasters', toolbar_location=None, tools="")

# Plot the data points
p.circle(x='population', y='disaster_count', source=final_country_pop_disaster_df, size=10, color='navy', alpha=0.5, legend_label='Data')

# Plot the regression line
p.line(x=regression_line_x, y=regression_line_y, line_width=2, color='red', legend_label='Regression Line')

# Add legend
p.legend.location = "top_left"


#save Javascript code and html code and show plot in browser
js_plot_nine, html_plot_nine = components(p)
show(p)